### 모델을 만들기 위한 코드입니다. 

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance

path = r"C:/Users/kyung/Desktop/UPT_dev/all_posnet_data/"

data1 = pd.read_csv(path+"all_data_1.csv")
data2 = pd.read_csv(path+"all_data_2.csv")
data3 = pd.read_csv(path+"all_data_3.csv")
data4 = pd.read_csv(path+"all_data_4.csv")

label = pd.read_excel(path+"all_data_label.xlsx", header = None, index_col = 1)

# 첫번째 컬럼빼고 다음에 나오는 컬럼들 삭제
drop_idx1 = []
for i in range(0, len(data1)-4, 4):
    drop_idx1.append(i+3)
    
drop_idx2 = []
for i in range(0, len(data2)-4, 4):
    drop_idx2.append(i+3)
    
drop_idx3 = []
for i in range(0, len(data3)-4, 4):
    drop_idx3.append(i+3)
    
drop_idx4 = []
for i in range(0, len(data4)-4, 4):
    drop_idx4.append(i+3)

data1.drop(data1.index[drop_idx1], inplace = True)
data1.reset_index(drop=True, inplace = True)

data2.drop(data2.index[drop_idx2], inplace = True)
data2.reset_index(drop=True, inplace = True)

data3.drop(data3.index[drop_idx3], inplace = True)
data3.reset_index(drop=True, inplace = True)

data4.drop(data4.index[drop_idx4], inplace = True)
data4.reset_index(drop=True, inplace = True)

# 4개 csv 파일 합치기
frames = [data1, data2, data3, data4]
data = pd.concat(frames, axis = 0)

# 합친 데이터 인덱스 재설정 및 데이터 타입 변환
data.reset_index(drop=True, inplace = True)
data = data.apply(pd.to_numeric)

# posenet score 모을 리스트 생성
score_idx = []
for i in range(0, len(data), 3):
    score_idx.append(i+2)
    
# score점수가 0.5이하면 버리기 -> NaN값으로 처리
columns = data.columns.to_list()
for idx in score_idx:  
    for column in columns:    
        if data.loc[idx][column] < 0.5:
            data.loc[idx-2][column] = np.nan # x  'NaN'
            data.loc[idx-1][column] = np.nan # y
            
# score data만 떼어내기 -> 2346row
score_data = data.loc[score_idx , :]
score_data.reset_index(drop=True, inplace = True)

# x, y 좌표만 남기기
data = data.drop(data.index[score_idx])
data.reset_index(drop=True, inplace = True)

# x좌표 index, y좌표 index 용 리스트 생성
x_idx = []
y_idx = []
for idx in range(0, len(data), 2):
    x_idx.append(idx)
    y_idx.append(idx+1)
    
# x_data, y_data분리
x_data = data.loc[x_idx, :]
x_data.reset_index(drop=True, inplace = True)

y_data = data.loc[y_idx, :]
y_data.reset_index(drop=True, inplace = True)

# 컬럼명 바꾸기용 리스트 생성
x_col = []
y_col = []
for column in columns:
    x_col.append(column+'_x')
    y_col.append(column+'_y')
    
# 컬럼명 바꾸기 
x_data.columns = x_col
y_data.columns = y_col

# 라벨링한 csv파일 전처리 
value = {'classification':label[2]}
label = pd.DataFrame(value, index = label.index)
label.reset_index(drop=True, inplace = True)

# 모두 합치기
all_data = pd.concat([x_data, y_data, label], axis = 1)

# 각 행의 모든 컬럼에서 null값이 20개 이상인 행 제거 => 1419rows
data_20 = all_data.drop(all_data.index[all_data.isnull().sum(axis = 1)>19])
data_20.reset_index(drop=True, inplace = True)

# 모든 특성 학습시키기 
X = data_20[data_20.columns[:34]]
y = data_20[data_20.columns[-1]]

#데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 66)

#모델 학습
model = XGBClassifier(
        learning_rate =0.09,
        n_estimators=110,
        max_depth=7,
        min_child_weight=1,
        gamma=0.3,
        reg_alpha=1e-05,
        subsample=0.61,
        colsample_bytree=0.7,
        seed = 150
) 
model.fit(X_train, y_train)

#평가 
print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도 : {:.3f}".format(model.score(X_test, y_test)))

훈련 세트 정확도: 0.998
테스트 세트 정확도 : 0.654


### 만든 모델을 picke로 저장하는 코드입니다. 

In [3]:
import pickle

In [4]:
# 모델 저장하기
pickle.dump(model, open("upt_model.pickle.dat", "wb"))

### pickle로 저장한 모델을 가져오는 코드입니다. 

In [5]:
loaded_model = pickle.load(open("upt_model.pickle.dat", "rb"))

### 예측할 사진의 데이터를 만드는 코드입니다. 

df_data는 data = pd.read_csv(path+"all_data_1.csv")와 같은 형태입니다. 

In [6]:
def make_data(df_data):
    # 첫번째 컬럼빼고 다음에 나오는 부위컬럼들 삭제
    drop_idx = []
    for i in range(0, len(df_data)-4, 4):
        drop_idx.append(i+3)
    
    df_data.drop(df_data.index[drop_idx], inplace = True)
    df_data.reset_index(drop=True, inplace = True)
    
    # 모든 값 str -> float형태로 
    df_data = df_data.apply(pd.to_numeric)
    
    # posenet score 모을 리스트 생성
    score_idx = []
    for i in range(0, len(df_data), 3):
        score_idx.append(i+2)
        
    # score점수가 0.5이하면 버리기 -> NaN값으로 처리
    columns = df_data.columns.to_list()
    for idx in score_idx:  
        for column in columns:    
            if df_data.loc[idx][column] < 0.5:
                df_data.loc[idx-2][column] = np.nan # x  'NaN'
                df_data.loc[idx-1][column] = np.nan # y
                
    # x, y 좌표만 남기기
    df_data = df_data.drop(df_data.index[score_idx])
    df_data.reset_index(drop=True, inplace = True)
    
    # x좌표 index, y좌표 index 용 리스트 생성
    x_idx = []
    y_idx = []
    for idx in range(0, len(df_data), 2):
        x_idx.append(idx)
        y_idx.append(idx+1)
        
    # x_data, y_data분리
    x_data = df_data.loc[x_idx, :]
    x_data.reset_index(drop=True, inplace = True)

    y_data = df_data.loc[y_idx, :]
    y_data.reset_index(drop=True, inplace = True)
    
    # 컬럼명 바꾸기용 리스트 생성
    x_col = []
    y_col = []
    for column in columns:
        x_col.append(column+'_x')
        y_col.append(column+'_y')
        
    # 컬럼명 바꾸기 
    x_data.columns = x_col
    y_data.columns = y_col
    
    # 모두 합치기
    all_data = pd.concat([x_data, y_data], axis = 1)
    all_data = all_data.apply(pd.to_numeric)
    
    return all_data #이 결과를 

---
이 아래부터는 예시입니다. 
### make_data함수가 제대로 작동하는지 확인하기위한 예시 코드입니다.

In [10]:
# 처리하기 전 단계인 csv 
test_data = pd.read_csv(path+"all_data_1.csv")
test_data

,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
0,0,0.007,0.008,0.033,0.056,0.146,0.16,0.191,0.509,0.174,0.965,0.265,0.458,0.181,0.158,0.153,1
1,0,0.048,0.174,0.492,0.334,0.195,0.211,0.824,0.425,1.467,0.943,0.354,0.886,0.987,0.97,1.344,1
2,0.941,0.271,0.966,0.104,0.964,0.469,0.968,0.928,0.855,0.774,0.82,0.057,0.08,0.125,0.133,0.067,0.096
3,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
4,0,0.019,0.07,0.255,0.374,0.821,0.733,1.004,0.913,4.399,0.88,1.33,1.168,0.991,0.949,1.018,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762,0.855,0.63,0.725,0.251,0.729,0.641,0.961,0.55,0.729,0.708,0.808,0.168,0.218,0.194,0.186,0.097,0.078
1763,nose,leftEye,rightEye,leftEar,rightEar,leftShoulder,rightShoulder,leftElbow,rightElbow,leftWrist,rightWrist,leftHip,rightHip,leftKnee,rightKnee,leftAnkle,rightAnkle
1764,0,0.023,0.06,0.229,0.115,1.124,0.67,3.579,1.068,6.685,1.005,0.137,0.108,3.396,1.216,0.949,1
1765,0,0.092,0.036,0.228,0.125,0.148,0.023,0.225,0.521,0.59,1.025,1.238,1.22,0.2,0.51,1.028,1


### DataFrame으로 불러온 csv파일을 make_data함수에 넣으면

In [11]:
result = make_data(test_data)
result

,nose_x,leftEye_x,rightEye_x,leftEar_x,rightEar_x,leftShoulder_x,rightShoulder_x,leftElbow_x,rightElbow_x,leftWrist_x,...,leftElbow_y,rightElbow_y,leftWrist_y,rightWrist_y,leftHip_y,rightHip_y,leftKnee_y,rightKnee_y,leftAnkle_y,rightAnkle_y
0,0.0,NaN,0.008,NaN,0.056,NaN,0.160,0.191,0.509,0.174,...,0.824,0.425,1.467,0.943,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.019,NaN,0.255,NaN,0.821,0.733,1.004,NaN,NaN,...,0.520,NaN,NaN,NaN,NaN,NaN,0.545,0.469,1.053,1.0
2,0.0,NaN,NaN,0.226,NaN,2.048,2.019,NaN,1.618,NaN,...,NaN,0.514,NaN,1.028,0.124,NaN,0.526,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,0.033,NaN,0.116,NaN,0.256,0.207,0.975,NaN,NaN,...,0.274,NaN,NaN,NaN,NaN,NaN,0.641,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,0.0,0.148,0.092,NaN,0.366,0.511,1.137,1.583,1.289,1.519,...,0.501,0.514,0.947,0.886,NaN,NaN,NaN,NaN,NaN,NaN
438,0.0,0.078,0.087,0.000,NaN,1.174,0.809,1.241,1.086,NaN,...,0.418,0.475,NaN,NaN,NaN,NaN,0.121,0.395,0.998,1.0
439,0.0,NaN,0.001,NaN,0.004,0.124,0.201,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440,0.0,0.005,0.001,NaN,0.055,0.124,0.165,0.186,0.207,0.989,...,1.151,0.959,1.014,1.028,NaN,NaN,NaN,NaN,NaN,NaN


### ↑ 생성한 모델에 넣을 수 있는 형태로 처리된 결과입니다. 

### pickle파일로 저장해놓은 모델을 불러오면

In [12]:
import pickle

loaded_model = pickle.load(open("upt_model.pickle.dat", "rb"))

### 따로 변수로 만들어놓은 result를 넣어도되고, make_data(test_data)를 넣어도됩니다.

In [13]:
y_pred = loaded_model.predict(result) 
predictions = [round(value) for value in y_pred]
print(predictions) 

[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### 예시 데이터의 결과는 리스트로 1,0값이 출력됩니다. 